In [1]:
import networkx as nx
import matplotlib as plt
import numpy as np
import pandas as pd
from itertools import permutations

In [105]:
def Spreadwave(wave_DF):
    waveList=[]
    df=wave_DF.sort_values(by=["waveUsage",'wave'],ascending=[True,True]) 
    df.reset_index(drop=True, inplace=True)#reset the index sequence
    #print(df)
    for i in range(len(df)):
        waveList.append(df["wave"][i])
        
    return waveList

def waveUpdate(Wave,Path,band):
    for i in range(band):
        wave_loc=Total_wave.index(Wave+i)
        wave_range[wave_loc]+=len(Path)
    #print(wave_range)
    #print(Total_wave)
    #print(randomseed)
    Wave_Infor={"wave":Total_wave,
               "waveUsage":wave_range
               }
    Wave_infor=pd.DataFrame(Wave_Infor)
    
    return Wave_infor
    

In [3]:
def zero_runs(a):
    # Create an array that is 1 where a is 0, and pad each end with an extra 0.
    iszero = np.concatenate(([0], np.equal(a, 0).view(np.int8), [0]))
    absdiff = np.abs(np.diff(iszero))
    # Runs start and end where absdiff is 1.
    ranges = np.where(absdiff == 1)[0].reshape(-1, 2)
    return ranges      

def pathBand(path):
    path_len=len(path)
    #print(path)
    #print(path[0][0])
    resource=Resource[path[0][0]][path[0][1]][path[0][2]]
    for i in range(1,path_len):
        resource=list(map(lambda x: x[0]+x[1], zip(resource, Resource[path[i][0]][path[i][1]][path[i][2]])))
    pathbandList=zero_runs(resource) 
    
    return pathbandList

In [21]:
def bypassType(path):
    path_len=len(path)
    if(path_len==1):
        bypass_type=0
    elif(path_len==2):
        edgeLoc0=edges_list.index((path[0][0],path[0][1]))
        edgeLoc1=edges_list.index((path[1][0],path[1][1]))
        if(path[1] in Out_bypassEdges[edgeLoc0][path[0][2]] and path[0] in In_bypassEdges[edgeLoc1][path[1][2]] ) :
            bypass_type=1
        else:
            bypass_type=0
            
    elif(path_len==3):       
        edgeLoc0=edges_list.index((path[0][0],path[0][1]))
        edgeLoc1=edges_list.index((path[1][0],path[1][1]))
        edgeLoc2=edges_list.index((path[2][0],path[2][1]))
        if(path[1] in Out_bypassEdges[edgeLoc0][path[0][2]] and path[0] in In_bypassEdges[edgeLoc1][path[1][2]] ) :
            bypass_type=1
        elif(path[2] in Out_bypassEdges[edgeLoc0][path[1][2]] and path[1] in In_bypassEdges[edgeLoc1][path[2][2]] ):
            bypass_type=1
        else:
            bypass_type=0
                       
            
    return bypass_type
                
                
            

In [5]:
def pathBand_Ocuupy(loc1,loc2,wave1,band,index):
    path=SD_pathLink[loc1][loc2]
    pathBand=ALL_SD_bandwidth[loc1][loc2]
    for i in range(len(path)):
        for j in range(band):
            #print(path)
            #print(wave1)
            #print(Resource[path[i][0]][path[i][1]][path[i][2]][wave1+j])
            Resource[path[i][0]][path[i][1]][path[i][2]][wave1+j]+=index+1
            
    return

        
def pathUpdate():
    SR_bandwidth=[]
    #MaxBand=[]
    Hops=[]
    for i in range(len(SD_pathLink)):
        band=[]
        #Max=[]
        hops=[]        
        for j in range(len(SD_pathLink[i])):
            #cost.append(pathCost(SD_pathLink[i][j]))
            hops.append(len(SD_pathLink[i][j]))
            b=pathBand(SD_pathLink[i][j])
            #Max.append(max(b[:,1] - b[:,0]))
            band.append(b)            
        #SR_pathLink_cost.append(cost)
        SR_bandwidth.append(band)
        #MaxBand.append(Max)
        Hops.append(hops)
    global ALL_SD_bandwidth
    ALL_SD_bandwidth=SR_bandwidth
        
    return

In [6]:
def CoreSetup():
    Nodes_list=list(G.nodes)
    #Link_bypass=[]
    for i in Nodes_list:
        neighborList=list(G.neighbors(i))
        #bypasscore_num=len(neighborList)-1# set the bypass core number within each fiber 
        for j in range(len(neighborList)):
            dis=1
            while(j+dis<len(neighborList)):
                input_loc=edges_list.index((neighborList[j],i))
                output_loc=edges_list.index((i,neighborList[j+dis]))
                successFlag=0
                for m in range(7):
                    if(len(bypassEdges[input_loc][m])==0):
                        for n in range(7):
                            if(len(bypassEdges[output_loc][n])==0):
                                bypassEdges[output_loc][n].append(((neighborList[j],i,m)))
                                bypassEdges[input_loc][m].append(((i,neighborList[j+dis],n)))
                                successFlag=1
                                break
                        if(successFlag==1):
                            break
                input_loc=edges_list.index((neighborList[j+dis],i))
                output_loc=edges_list.index((i,neighborList[j]))
                successFlag=0
                for m in range(7):
                    if(len(bypassEdges[input_loc][m])==0):
                        for n in range(7):
                            if(len(bypassEdges[output_loc][n])==0):
                                bypassEdges[output_loc][n].append(((neighborList[j+dis],i,m)))
                                bypassEdges[input_loc][m].append(((i,neighborList[j],n)))
                                successFlag=1
                                break
                        if(successFlag==1):
                            break
                                                                                                        
                dis+=1
    return;

In [7]:
def edgeToLinks(path):
    pathlist=[]
    path_len=len(path)
    corePossib=7**path_len
    for i in range(corePossib):
        List=[]
        List.append((path[0]+(i%7,)))
        edgeLoc=edges_list.index(path[0])
        
        if(len(In_bypassEdges[edgeLoc][i%7])>0):
            List=[]
            continue
        for j in range(1,path_len):
            List.append((path[j]+((int(i/(7**j)%7)),)))
            edgeLoc0=edges_list.index(path[j-1])
            edgeLoc1=edges_list.index(path[j])
            if(len(Out_bypassEdges[edgeLoc0][List[j-1][2]])>0 and List[j] not in Out_bypassEdges[edgeLoc0][List[j-1][2]] ) :

                List=[]
                break
            if(len(In_bypassEdges[edgeLoc1][List[j][2]])>0 and List[j-1] not in In_bypassEdges[edgeLoc1][List[j][2]] ) :

                List=[]
                break
        edgeLoc=edges_list.index(path[path_len-1])
        if(len(Out_bypassEdges[edgeLoc][int(i/(7**(path_len-1)))])>0):            
            List=[]
            continue 
        if(len(List)>0):
            pathlist.append(List)    
    return pathlist

In [8]:
def PathAlgorithm_1(PathList,hopsList,bandwidth,pathBandList,bypassList,wave,list1,list2):
    Max=np.zeros(len(pathBandList))
    Sum=np.zeros(len(pathBandList))
    for i in range(len(pathBandList)):
        #print (pathBandList[i][:,1] - pathBandList[i][:,0])
        #print(i)
        #print(PathList[0])
        Max[i]=max(pathBandList[i][:,1] - pathBandList[i][:,0])
        Sum[i]=sum(pathBandList[i][:,1] - pathBandList[i][:,0])
    
    infor={"place":range(len(PathList)),
           "bypass":bypassList,
           "bandwidth": Max,
           "hops": hopsList,
           "sum":Sum
         }
    
    Path_infor=pd.DataFrame(infor)     
    #print(Path_infor)
    df=Path_infor.sort_values(by=list1,ascending=list2) 
    df.reset_index(drop=True, inplace=True)#reset the index sequence
    #print(df)
    Path=[]
    
    for i in range(len(df)):
        successful_flag=0
        if(df["bandwidth"][i]>bandwidth):
            for j in pathBandList[df["place"][i]]:
                if(wave>=j[0] and wave+bandwidth<=j[1]):
                    Path=PathList[df["place"][i]]
                    traffic_feedback.append(df["bypass"][i])
                    successful_flag=1
                    break 
        if(successful_flag==1):
            break
    if(len(Path)==0):
        traffic_feedback.append(-1)
        #print(df)
    return Path


In [9]:
def PathOccupy_percent():
    Bypass_2=[]
    Bypass_1=[]
    Bypass_0=[]
    for i in range(len(edges_list)):
        if(edges_list[i] in Traffic_SD_pairs):
            for j in range(len((SD_pathLink[i]))):
                percent=1-(sum(ALL_SD_bandwidth[i][j][:,1]-ALL_SD_bandwidth[i][j][:,0])-1)/160
                if(Bypass_Type[i][j]==2):
                    Bypass_2.append((percent))
                elif(Bypass_Type[i][j]==1):
                    Bypass_1.append((percent))
                elif(Bypass_Type[i][j]==0):
                    Bypass_0.append((percent))
    Bypass=[]
    Bypass.append(Bypass_0)
    Bypass.append(Bypass_1)
    
    Bypass.append(Bypass_2)
    
    return Bypass

In [10]:
def Link_percentage():
    Edge=[]
    Link=[]
    for i in edges_list:
        Edge.append(str(i))
    for j in range(7):        
        occupy=[]
        for i in edges_list:
            #print(i)
            #print(list(np.nonzero(Resource[i[0]][i[1]][j])))
            k=np.array(Resource[i[0]][i[1]][j])
            per=np.count_nonzero(k)/160
            occupy.append(per)
        Link.append(occupy)
    df=pd.DataFrame(Link,columns=Edge)
    #print(df)
    #df.rename()
    return df
        

In [11]:
def Path_percent():
    Bypass_2=[]
    Bypass_1=[]
    Bypass_0=[]
    for i in range(len(edges_list)):
        if(edges_list[i] in Traffic_SD_pairs):
            for j in range(len((SD_pathLink[i]))):
                #percent=1-(sum(ALL_SD_bandwidth[i][j][:,1]-ALL_SD_bandwidth[i][j][:,0])-1)/160
                if(Bypass_Type[i][j]==2):
                    Bypass_2.append((SD_pathLink[i][j]))
                elif(Bypass_Type[i][j]==1):
                    Bypass_1.append((SD_pathLink[i][j]))
                elif(Bypass_Type[i][j]==0):
                    Bypass_0.append((SD_pathLink[i][j]))
    Bypass=[]
    Bypass.append(Bypass_0)
    Bypass.append(Bypass_1)
    
    Bypass.append(Bypass_2)
    
    return Bypass

In [12]:
TrafficCost=0
node_list=[0,1,2,3]
Traffic_SD_pairs=[(0,3),(0,1),(0,2),(1,2),(2,3),(1,3),(2,1)]
SR_pair=[(0,1),(1,0),(1,2),(2,1),(0,2),(2,0),(2,3),(3,2),(0,3),(3,0),(1,3),(3,1)]
edges_list=[(0,1),(1,0),(1,2),(2,1),(0,2),(2,0),(2,3),(3,2),(1,3),(3,1)]
bypassEdges=[]
In_bypassEdges=[]
Out_bypassEdges=[]
for i in range(len(edges_list)):
    BY=[]
    BY0=[]
    BY1=[]
    
    for j in range(7):
        BY.append([])
        BY0.append([])
        BY1.append([])
    bypassEdges.append(BY)
    In_bypassEdges.append(BY0)
    Out_bypassEdges.append(BY1)
    

   
#先对D=2的节点进行bypass设置，在对D=3的节点进行bypass设置
#每次设置都在该link中不存在

G=nx.DiGraph()
G.add_nodes_from(node_list)
G.add_edges_from(edges_list)
nx.draw(G)

#plt.show()


link_list=[] 

for k in range(len(edges_list)):
    link_list0=[]
    for i in range(7):
        link_list0.append(edges_list[k]+(i,)) 
    link_list.append(link_list0)

SD_path=[]
SR_pathLink=[]
for i in range(len(SR_pair)):
    path0=[]
    for path in nx.all_simple_paths(G, source=SR_pair[i][0], target=SR_pair[i][1]):
        path_edges=[]            
        for j in range(len(path)-1):
            path_edges.append((path[j],path[j+1]))
        path0.append(path_edges)
    SD_path.append(path0)


CoreSetup() 
for i in range(len(bypassEdges)):
    Link=bypassEdges[i]
    for j in range(7):
        if(len(Link[j])>0):
            if(Link[j][0][0]==edges_list[i][1]):
                
                Out_bypassEdges[i][j]=Link[j]
            elif(Link[j][0][1]==edges_list[i][0]):
                In_bypassEdges[i][j]=Link[j]
                

In [13]:
SD_pathLink=[]  
for i in range(len(SD_path)):
    Links=[]
    for j in range(len(SD_path[i])):
        Links=Links+edgeToLinks(SD_path[i][j])
        
    SD_pathLink.append(Links)


In [14]:
Tag_range = list(permutations(range(3), 3))
Tag_list=["hops","bypass","sum"]
FalseorTrue=[True, True, False]
for i in range(8):
    k1=int(i%2)
    k2=int((i%4)/2)
    k3=int(i/4)
    FalseorTrue.append((k1,k2,k3))
    
print(FalseorTrue)

FalseTrueList=[False,True]


[(0, 0, 0), (1, 0, 0), (0, 1, 0), (1, 1, 0), (0, 0, 1), (1, 0, 1), (0, 1, 1), (1, 1, 1)]


In [130]:
Success_list=[]
Tag_List=[]
True_List=[]
for tag_selection in range(1):
    for true_selection in range(1):
    
    
        #Tag=Tag_range[tag_selection]
        #choose=FalseorTrue[true_selection]
        list1= ['hops', 'bypass', 'sum']
        list2= [True, False, True]
        #for i in range(len(Tag)):
            #list1.append(Tag_list[Tag[i]])
        #for i in range(len(choose)):
            #list2.append(FalseTrueList[choose[i]])
        Total_bandwidth=161
        Resource=np.zeros([4,4,7,161]).tolist()
        ALL_SD_bandwidth=[]
        MaxBand=[]
        Hops=[]
        Bypass_Type=[]
        for i in range(len(SD_pathLink)):
            Band=[]
            Max=[]
            Hhops=[]
            Bypass=[]
            for j in range(len(SD_pathLink[i])):
                Hhops.append(len(SD_pathLink[i][j]))
                b=pathBand(SD_pathLink[i][j])
                Max.append(max(b[:,1] - b[:,0]))
                Band.append(b)
                Bypass.append(bypassType(SD_pathLink[i][j]))
            ALL_SD_bandwidth.append(Band)
            MaxBand.append(Max)
            Hops.append(Hhops)
            Bypass_Type.append(Bypass)
        
        Total_wave=list(range(Total_bandwidth-1))
        wave_range=list(np.zeros(len(Total_wave)))
        randomseed=[]
        for i in range(len(Total_wave)):
            randomseed.append(np.random.randint(len(Total_wave)))
        Wave_Infor={"wave":Total_wave,
                   "waveUsage":wave_range
                   }
        Wave_infor=pd.DataFrame(Wave_Infor)

        
        
        # In[214]:
        
        
        Traffic=pd.read_csv('traffic.csv',sep=',',names=['S','D','band'])
        SuccessFlag=np.zeros(len(Traffic))
        BypassFlag=np.zeros(len(Traffic))
        traffic_wave=[]
        traffic_path=[]
        traffic_feedback=[]
        Path_band_usage=[]
        Traffic_num=len(Traffic)
        for i in range(Traffic_num):
            traffic=(Traffic['S'][i],Traffic['D'][i])
            band=Traffic['band'][i]
            traffic_Path=SD_pathLink[SR_pair.index(traffic)]
            traffic_Hops=Hops[SR_pair.index(traffic)]
            traffic_Band=ALL_SD_bandwidth[SR_pair.index(traffic)]
            traffic_Bypass=Bypass_Type[SR_pair.index(traffic)]
            wave=Spreadwave(Wave_infor)
            #print(wave)
            #print(Wave_infor)
            for j in range(len(wave)):
                PathFlag=0
                select_wave=wave[j]
                print(select_wave)
                selectPath=PathAlgorithm_1(traffic_Path,traffic_Hops,band,traffic_Band,traffic_Bypass,select_wave,list1,list2)
                #print(selectPath)
                if(len(selectPath) and select_wave+band<=160):
                    Wave_infor=waveUpdate(select_wave,selectPath,band)
                    Pathloc=traffic_Path.index(selectPath)
                    pathBand_Ocuupy(SR_pair.index(traffic),Pathloc,select_wave,band,i)
                    PathFlag=1
                    traffic_wave.append(select_wave)
                    traffic_path.append(selectPath)            
                    pathUpdate()
                    if(PathFlag==1):
                        SuccessFlag[i]=1
                    break
                
        Success_list.append(SuccessFlag.sum()) 
        Tag_List.append(list1)
        True_List.append(list2)

0
1
5
7
8
10
11
15
19
20
24
26
30
32
36
38
40
42
43
45
46
47
51
52
54
55
57
58
60
62
63
64
65
69
70
74
78
80
82
83
84
85
86
90
91
95
99
101
102
104
106
107
108
112
114
116
117
118
119
123
127
128
129
130
134
136
138
139
143
144
146
147
151
152
153
154
155
156
157
158
159
0
157
158
4
7
8
12
16
18
19
20
21
22
24
25
29
30
32
33
35
39
45
51
52
53
54
55
57
60
61
65
67
69
70
72
73
74
75
77
78
79
80
81
82
90
92
93
95
96
98
99
101
107
112
113
114
115
117
119
123
125
126
128
129
133
136
138
142
147
152
153
154
156
0
1
2
3
7
8
10
12
13
14
15
19
21
25
26
28
29
30
34
38
39
43
45
49
51
52
54
58
60
61
62
64
68
72
75
76
78
79
80
82
83
87
91
92
96
97
98
101
105
108
112
113
114
116
117
118
120
121
122
124
126
129
130
134
136
137
138
139
140
141
142
146
147
148
149
151
152
156
157
158
159
1
5
6
7
9
11
13
14
15
16
17
21
22
23
25
28
30
31
32
33
34
38
40
43
45
46
47
51
54
56
58
59
60
64
65
66
68
69
70
71
75
76
80
81
82
83
85
89
93
97
98
102
105
107
111
114
115
116
118
119
121
125
126
127
129
134
135
137
14

43
47
48
49
51
52
53
54
58
62
66
71
73
74
77
78
73
77
91
93
97
99
100
106
107
109
115
121
125
121
127
129
133
134
140
144
148
149
150
154
157
158
159
0
157
158
159
4
5
6
7
8
159
4
5
6
7
10
11
13
18
20
21
20
27
28
30
33
34
38
47
38
51
58
59
60
63
66
67
63
66
63
71
63
75
63
82
83
85
87
88
93
94
88
93
97
100
101
105
107
111
113
115
116
117
119
120
121
122
124
125
133
124
125
139
144
145
147
148
149
156
124
125
139
144
139
148
149
0
2
4
5
6
11
4
6
17
27
29
30
33
35
36
37
38
42
44
45
46
47
49
44
45
46
47
53
54
47
53
56
57
58
57
61
65
66
82
65
84
87
88
92
96
105
106
107
108
113
105
117
118
120
117
126
127
128
129
130
131
132
138
126
127
128
129
130
131
132
142
143
148
126
127
128
129
130
131
132
127
129
130
131
142
143
129
131
142
153
0
2
3
6
7
6
10
11
12
20
21
20
22
26
27
28
29
30
27
29
35
29
37
29
41
45
48
56
58
59
60
61
62
64
58
59
60
61
62
68
71
72
75
76
81
82
84
85
86
58
60
61
68
71
72
75
76
71
72
75
81
82
84
85
101
102
104
105
109
117
119
124
125
126
129
130
131
137
138
140
141
147
152

159
4
6
7
9
12
19
20
21
22
25
27
28
29
30
35
36
37
44
53
54
55
57
58
59
67
75
76
82
83
84
85
86
91
92
97
98
108
112
113
116
118
119
124
126
127
128
136
137
139
140
141
144
145
147
148
150
153
158
0
1
5
8
10
15
17
23
24
31
32
38
39
41
42
49
51
52
62
63
64
65
68
69
74
77
78
93
94
95
96
100
101
102
103
104
105
106
110
120
129
130
131
132
146
151
154
11
16
18
40
47
50
56
70
13
14
26
14
33
14
43
48
14
43
60
43
66
81
87
66
81
88
81
89
90
99
89
90
109
89
90
114
90
115
117
121
122
123
133
90
115
117
121
122
123
135
138
142
149
152
159
4
6
7
9
12
13
14
19
20
21
22
25
26
27
28
149
152
159
4
6
7
9
159
4
6
7
13
14
19
14
21
22
21
25
26
27
33
34
35
36
37
43
27
33
34
35
36
37
48
33
35
36
35
37
35
53
54
55
57
58
59
60
61
66
35
53
54
58
59
60
61
75
59
60
82
83
84
85
82
83
84
83
89
90
92
97
98
108
112
113
114
115
83
89
90
92
97
98
108
112
113
114
119
123
89
90
92
97
98
108
89
90
92
97
112
113
114
119
112
113
114
127
128
133
113
114
127
128
137
138
140
141
142
143
144
145
147
148
149
150
153
158
0
114
12

31
40
45
46
50
52
54
55
70
76
78
101
102
119
124
129
137
2
51
56
71
72
73
127
135
152
125
132
138
132
143
144
143
146
147
158
159
0
143
146
147
158
159
4
5
6
7
10
18
19
22
23
24
25
27
143
146
147
143
146
158
159
4
5
6
7
10
18
19
22
23
24
25
33
34
35
36
37
41
43
44
47
57
59
60
61
66
67
68
75
88
89
92
93
96
97
104
107
108
109
111
112
113
114
118
120
121
131
132
142
143
145
146
149
150
151
153
154
155
156
158
159
0
3
4
8
9
11
5
6
7
10
18
19
22
23
24
25
33
34
35
36
37
41
43
44
47
57
59
60
61
66
67
68
75
88
89
92
93
96
97
104
107
108
109
111
112
113
114
118
120
121
131
132
142
143
145
146
149
150
151
153
154
155
156
158
159
0
3
4
8
9
16
17
20
26
32
38
39
42
48
49
53
58
62
63
64
65
69
74
77
79
80
81
82
83
84
85
86
87
90
91
94
95
98
99
100
103
105
106
110
115
116
117
122
123
126
128
130
133
134
136
138
139
140
141
144
147
157
1
15
21
27
29
30
31
40
45
46
50
52
54
55
70
76
78
101
102
119
124
129
137
2
11
12
14
28
51
56
71
72
73
127
135
148
152
13
125
5
7
10
18
19
22
23
24
25
33
34
35
36
37
41


2
4
10
11
12
14
17
19
20
28
38
39
42
47
49
51
56
65
66
71
72
73
74
75
77
90
91
93
98
127
133
134
135
143
147
148
152
157
158
13
18
21
36
40
41
48
50
76
125
150
53
58
61
62
63
64
67
68
69
79
80
81
82
83
61
62
63
64
67
68
69
79
80
81
82
85
86
87
95
96
99
100
103
104
105
106
107
108
61
62
63
64
67
68
69
79
80
81
82
85
86
87
95
96
99
100
103
104
105
106
107
110
111
112
113
114
115
61
63
64
67
64
68
69
68
79
68
80
81
82
85
86
87
95
96
99
100
103
104
105
106
107
110
111
112
113
114
117
80
81
82
85
86
87
95
96
99
100
103
104
105
106
107
110
111
112
113
114
121
122
123
126
128
130
136
138
139
140
141
144
154
0
1
3
5
6
7
8
9
15
16
22
23
24
25
26
27
29
30
31
32
33
34
35
37
43
44
45
46
52
53
54
55
57
59
60
61
62
63
64
67
70
78
79
88
89
92
94
97
101
102
108
115
116
117
118
119
120
124
129
131
132
137
142
145
146
149
151
153
155
156
159
2
4
10
11
12
14
17
19
20
28
38
39
42
47
49
51
56
58
65
66
68
71
72
73
74
75
77
83
84
90
91
93
98
109
127
133
134
135
143
147
148
152
157
158
13
18
21
36
40
41
48
50

...

60
61
64
65
66
67
68
69
76
77
78
80
82
83
85
86
87
88
90
91
92
93
94
96
97
98
99
100
103
104
105
108
109
111
112
113
115
116
117
118
121
123
125
127
128
129
130
131
132
133
134
137
138
139
140
141
143
145
147
148
150
156
159
3
4
5
6
9
10
17
19
22
23
25
26
32
37
38
39
40
43
44
47
51
52
57
62
63
73
74
75
79
81
84
95
101
106
110
114
119
122
126
135
136
151
152
153
158
2
11
12
13
24
29
30
33
34
70
71
72
89
102
107
120
15
16
18
124
142
144
146
149
154
155
157
0
1
7
8
14
20
21
27
28
31
35
36
41
42
45
46
48
49
50
53
54
55
56
58
59
60
61
64
65
66
67
68
69
76
77
78
80
82
83
85
86
87
88
90
91
92
93
94
96
97
98
99
100
103
104
105
108
109
111
112
113
115
116
117
118
121
123
125
127
128
129
130
131
132
133
134
137
138
139
140
141
143
145
147
148
150
156
159
3
4
5
6
9
10
17
19
22
23
25
26
32
37
38
39
40
43
44
47
51
52
57
62
63
73
74
75
79
81
84
95
101
106
110
114
119
122
126
135
136
151
152
153
158
2
11
12
13
24
29
30
33
34
70
71
72
89
102
107
120
15
16
18
124
142
144
146
149
154
155
157
0
1
7
8
14


83
85
86
87
88
90
91
92
93
94
96
97
98
99
100
104
105
108
109
111
112
113
115
116
117
118
121
123
125
127
128
129
130
131
132
133
134
137
138
139
140
141
143
144
145
146
147
148
150
156
1
3
4
5
6
8
9
10
17
19
22
23
25
26
27
32
37
38
39
40
43
44
45
47
48
51
52
54
55
57
62
63
73
74
75
79
81
84
95
101
106
110
114
119
122
126
135
136
142
149
151
152
153
154
158
0
2
11
12
13
14
24
29
30
33
34
60
61
70
71
72
89
102
103
107
120
15
16
18
124
159
155
157
7
20
21
28
31
35
36
41
42
46
49
50
53
56
58
59
64
65
66
67
68
69
76
77
78
80
82
83
85
86
87
88
90
91
92
93
94
96
97
98
99
100
104
105
108
109
111
112
113
115
116
117
118
121
123
125
127
128
129
130
131
132
133
134
137
138
139
140
141
143
144
145
146
147
148
150
156
1
3
4
5
6
8
9
10
17
19
22
23
25
26
27
32
37
38
39
40
43
44
45
47
48
51
52
54
55
57
62
63
73
74
75
79
81
84
95
101
106
110
114
119
122
126
135
136
142
149
151
152
153
154
158
0
2
11
12
13
14
24
29
30
33
34
60
61
70
71
72
89
102
103
107
120
15
16
18
124
159
155
157
7
20
7
21
28
31
35
3

138
139
140
141
143
144
145
146
147
148
150
155
156
157
1
3
4
5
6
8
9
10
17
19
23
25
26
27
32
36
37
38
39
40
42
43
44
45
47
48
50
52
54
55
56
57
62
63
64
73
74
75
79
81
84
95
101
106
110
114
119
122
126
135
136
142
149
151
152
153
154
158
0
2
7
11
12
13
14
20
21
24
28
29
30
31
33
34
35
41
49
51
53
60
61
66
67
70
71
72
89
102
103
107
120
15
16
18
22
124
159
46
58
59
65
68
69
76
77
78
80
82
83
85
86
87
88
90
91
92
93
94
96
97
98
99
100
104
105
108
109
111
112
113
115
116
117
118
121
123
125
127
128
129
130
131
132
133
134
137
138
139
140
141
143
144
145
146
147
148
150
155
156
157
1
3
4
5
6
8
9
10
17
19
23
25
26
27
32
36
37
38
39
40
42
43
44
45
47
48
50
52
54
55
56
57
62
63
64
73
74
75
79
81
84
95
101
106
110
114
119
122
126
135
136
142
149
151
152
153
154
158
0
2
7
11
12
13
14
20
21
24
28
29
30
31
33
34
35
41
49
51
53
60
61
66
67
70
71
72
89
102
103
107
120
15
16
18
22
124
159
46
58
46
59
46
65
68
69
76
77
78
80
82
83
85
86
87
88
90
91
92
93
94
96
97
98
65
68
65
76
77
78
80
82
83
85
86


20
21
24
28
29
30
31
33
34
35
41
49
51
53
60
61
66
67
70
71
72
76
77
78
80
86
87
89
90
92
94
97
98
99
100
102
103
107
111
112
120
9
10
15
16
18
22
124
134
159
91
93
104
105
109
113
115
116
117
118
121
123
125
127
128
129
130
131
132
133
137
138
139
140
141
143
144
145
146
147
148
150
155
156
157
1
3
4
5
6
8
17
19
23
25
26
27
32
36
37
38
39
40
42
43
44
45
46
47
48
50
52
54
55
56
57
58
59
62
63
64
65
68
69
73
74
75
79
81
82
83
84
85
88
95
96
101
106
108
110
114
119
122
126
135
136
142
149
151
152
153
154
158
0
2
7
11
12
13
14
20
21
24
28
29
30
31
33
34
35
41
49
51
53
60
61
66
67
70
71
72
76
77
78
80
86
87
89
90
92
94
97
98
99
100
102
103
107
111
112
120
9
10
15
16
18
22
124
134
159
91
93
104
105
109
113
115
116
117
118
121
123
125
127
128
129
130
131
132
133
137
138
139
140
141
143
144
145
146
147
148
150
155
156
157
1
3
4
5
6
8
17
19
23
25
26
27
32
36
37
38
39
40
42
43
44
45
46
47
48
50
52
54
55
56
57
58
59
62
63
64
65
68
69
73
74
75
79
81
82
83
84
85
88
95
96
101
106
108
110
114
119
12

102
103
105
107
109
111
112
115
120
128
129
9
10
15
16
18
22
106
124
134
159
127
130
131
130
132
133
137
138
139
140
141
143
144
145
146
147
148
150
155
156
157
1
3
4
5
6
8
17
19
23
25
26
27
32
36
37
38
39
40
42
43
44
45
46
47
48
50
52
54
55
56
57
58
59
62
63
64
65
68
69
73
74
75
79
81
82
83
84
85
88
91
93
95
96
101
104
108
110
113
114
116
117
118
119
121
122
123
125
126
127
131
135
136
142
149
151
152
153
154
158
0
2
7
11
12
13
14
20
21
24
28
29
30
31
33
34
35
41
49
51
53
60
61
66
67
70
71
72
76
77
78
80
86
87
89
90
92
94
97
98
99
100
102
103
105
107
109
111
112
115
120
128
129
9
10
15
16
18
22
106
124
134
159
130
132
133
137
138
139
140
141
143
144
145
146
147
148
150
155
156
157
1
3
4
5
6
8
17
19
23
25
26
27
32
36
37
38
39
40
42
43
44
45
46
47
48
50
52
54
55
56
57
58
59
62
63
64
65
68
69
73
74
75
79
81
82
83
84
85
88
91
93
95
130
132
133
137
138
139
140
141
143
144
145
146
147
148
150
155
156
157
1
3
4
5
6
8
17
19
23
25
26
27
32
36
37
38
39
40
42
43
44
45
46
47
48
50
52
54
55
56
57


In [131]:
print(Success_list)

[3712.0]


In [132]:
print(Link_percentage())

    (0, 1)   (1, 0)   (1, 2)   (2, 1)   (0, 2)   (2, 0)   (2, 3)   (3, 2)  \
0  0.91250  0.83125  0.94375  0.90625  0.83125  0.91250  0.83750  0.92500   
1  0.94375  0.90625  0.90000  0.81250  0.93125  0.90000  0.94375  0.93125   
2  0.87500  0.89375  0.90000  0.93125  0.94375  0.93125  0.86250  0.81250   
3  0.95625  0.88750  0.83750  0.92500  0.85000  0.90000  0.88750  0.90000   
4  0.94375  0.91250  0.90000  0.89375  0.94375  0.90625  0.90625  0.88750   
5  0.91875  0.83750  0.86250  0.92500  0.86875  0.88125  0.88125  0.95625   
6  0.90625  0.90000  0.88125  0.83125  0.90000  0.85625  0.84375  0.87500   

    (1, 3)   (3, 1)  
0  0.87500  0.89375  
1  0.81250  0.90000  
2  0.81250  0.86250  
3  0.88125  0.92500  
4  0.82500  0.93125  
5  0.86875  0.92500  
6  0.91875  0.91250  


In [133]:
occupy=0
for j in range(7):        
    
    for i in edges_list:
            #print(i)
            #print(list(np.nonzero(Resource[i[0]][i[1]][j])))
        k=np.array(Resource[i[0]][i[1]][j])
        S=np.count_nonzero(k)
        occupy+=S
print(occupy)
print(occupy/(7*10))

9988
142.68571428571428


In [134]:
k=0
for i in traffic_path:
    k+=len(i)
print(k/len(traffic_path))

1.3771551724137931


In [135]:
k=0
for i in range(Traffic_num):
    if(SuccessFlag[i]==1):
        k+=Traffic['band'][i]
print(k)

7068


In [16]:

print(Success_list)

[1230.0, 1357.0, 1223.0, 1355.0, 1213.0, 1371.0, 1200.0, 1362.0, 1234.0, 1354.0, 1210.0, 1378.0, 1241.0, 1359.0, 1194.0, 1362.0, 1315.0, 1225.0, 1358.0, 1241.0, 1326.0, 1212.0, 1361.0, 1214.0, 1347.0, 1234.0, 1314.0, 1212.0, 1345.0, 1240.0, 1307.0, 1212.0, 1337.0, 1227.0, 1347.0, 1230.0, 1333.0, 1224.0, 1338.0, 1225.0, 1343.0, 1233.0, 1337.0, 1224.0, 1343.0, 1227.0, 1338.0, 1224.0]


In [17]:
print(Tag_List[Success_list.index(max(Success_list))])
print(True_List[Success_list.index(max(Success_list))])

['hops', 'sum', 'bypass']
[True, True, False]


In [20]:
print(max(Success_list))

1378.0


In [19]:
print(Link_percentage())

    (0, 1)   (1, 0)   (1, 2)   (2, 1)   (0, 2)   (2, 0)   (2, 3)   (3, 2)  \
0  0.91250  0.92500  0.93125  0.92500  0.92500  0.91250  0.93125  0.92500   
1  0.93125  0.92500  0.96875  0.88750  0.94375  0.92500  0.86250  0.93750   
2  0.95625  0.95000  0.92500  0.94375  0.86250  0.93750  0.90625  0.88125   
3  0.88750  0.86875  0.93125  0.92500  0.92500  0.86250  0.88125  0.88125   
4  0.94375  0.85625  0.89375  0.80625  0.87500  0.88750  0.90625  0.88750   
5  0.93750  0.82500  0.80625  0.85000  0.84375  0.86875  0.80000  0.87500   
6  0.89375  0.88750  0.85000  0.86250  0.91250  0.89375  0.88750  0.86875   

    (1, 3)   (3, 1)  
0  0.95625  0.95000  
1  0.88750  0.96875  
2  0.88125  0.90625  
3  0.85625  0.85000  
4  0.87500  0.92500  
5  0.87500  0.89375  
6  0.86250  0.90625  


In [52]:
import heapq


a=[1230.0, 1357.0, 1223.0, 1355.0, 1213.0, 1371.0, 1200.0, 1362.0, 1234.0, 1354.0, 1210.0, 1378.0, 1241.0, 1359.0, 1194.0, 1362.0, 1315.0, 1225.0, 1358.0, 1241.0, 1326.0, 1212.0, 1361.0, 1214.0, 1347.0, 1234.0, 1314.0, 1212.0, 1345.0, 1240.0, 1307.0, 1212.0, 1337.0, 1227.0, 1347.0, 1230.0, 1333.0, 1224.0, 1338.0, 1225.0, 1343.0, 1233.0, 1337.0, 1224.0, 1343.0, 1227.0, 1338.0, 1224.0]
tag=[['hops', 'bypass', 'sum'], ['hops', 'bypass', 'sum'], ['hops', 'bypass', 'sum'], ['hops', 'bypass', 'sum'], ['hops', 'bypass', 'sum'], ['hops', 'bypass', 'sum'], ['hops', 'bypass', 'sum'], ['hops', 'bypass', 'sum'], ['hops', 'sum', 'bypass'], ['hops', 'sum', 'bypass'], ['hops', 'sum', 'bypass'], ['hops', 'sum', 'bypass'], ['hops', 'sum', 'bypass'], ['hops', 'sum', 'bypass'], ['hops', 'sum', 'bypass'], ['hops', 'sum', 'bypass'], ['bypass', 'hops', 'sum'], ['bypass', 'hops', 'sum'], ['bypass', 'hops', 'sum'], ['bypass', 'hops', 'sum'], ['bypass', 'hops', 'sum'], ['bypass', 'hops', 'sum'], ['bypass', 'hops', 'sum'], ['bypass', 'hops', 'sum'], ['bypass', 'sum', 'hops'], ['bypass', 'sum', 'hops'], ['bypass', 'sum', 'hops'], ['bypass', 'sum', 'hops'], ['bypass', 'sum', 'hops'], ['bypass', 'sum', 'hops'], ['bypass', 'sum', 'hops'], ['bypass', 'sum', 'hops'], ['sum', 'hops', 'bypass'], ['sum', 'hops', 'bypass'], ['sum', 'hops', 'bypass'], ['sum', 'hops', 'bypass'], ['sum', 'hops', 'bypass'], ['sum', 'hops', 'bypass'], ['sum', 'hops', 'bypass'], ['sum', 'hops', 'bypass'], ['sum', 'bypass', 'hops'], ['sum', 'bypass', 'hops'], ['sum', 'bypass', 'hops'], ['sum', 'bypass', 'hops'], ['sum', 'bypass', 'hops'], ['sum', 'bypass', 'hops'], ['sum', 'bypass', 'hops'], ['sum', 'bypass', 'hops']]
true=[[False, False, False], [True, False, False], [False, True, False], [True, True, False], [False, False, True], [True, False, True], [False, True, True], [True, True, True], [False, False, False], [True, False, False], [False, True, False], [True, True, False], [False, False, True], [True, False, True], [False, True, True], [True, True, True], [False, False, False], [True, False, False], [False, True, False], [True, True, False], [False, False, True], [True, False, True], [False, True, True], [True, True, True], [False, False, False], [True, False, False], [False, True, False], [True, True, False], [False, False, True], [True, False, True], [False, True, True], [True, True, True], [False, False, False], [True, False, False], [False, True, False], [True, True, False], [False, False, True], [True, False, True], [False, True, True], [True, True, True], [False, False, False], [True, False, False], [False, True, False], [True, True, False], [False, False, True], [True, False, True], [False, True, True], [True, True, True]]
big_index=list(map(a.index,heapq.nlargest(5,a)))
print(len(true))
print(big_index)
for i in big_index:
    print(a[i])
    print(tag[i])
    print(true[i])

48
[11, 5, 7, 7, 22]
1378.0
['hops', 'sum', 'bypass']
[True, True, False]
1371.0
['hops', 'bypass', 'sum']
[True, False, True]
1362.0
['hops', 'bypass', 'sum']
[True, True, True]
1362.0
['hops', 'bypass', 'sum']
[True, True, True]
1361.0
['bypass', 'hops', 'sum']
[False, True, True]
